# Algorithms 4, 5, & 6

In [75]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import trapezoid

### Algorithm 4 - Feature Extraction

In [80]:
# The dataframe may be different columns so adjust as needed based on other algorithms

def feature_engineering(data, threshold):
    high_peak = np.max(data["CH1"])
    low_peak = np.min(data["CH1"])
    evoked_potentials = data["CH1"]

    # Calculate Peak to Peak amplitude
    # TODO: This might still need to be normalized
    distance = np.subtract(high_peak, low_peak)
    p2pa = np.abs(distance)

    # Activation Latency
    # TODO: Check non-zero indices variable
    # Calculate Stimulus onset: 1st time when stimulus != 0
    stim_data = np.array(data[1])
    non_zero_indices = np.nonzero(stim_data)[0]  # Get indices of non-zero elements
    stim_onset = non_zero_indices[0] if non_zero_indices.size > 0 else None
    # Calculate Activation onset: 1st time when evoke potential != 0
    act_data = np.array(data[2])
    non_zero_indices = np.nonzero(stim_data)[0]  # Get indices of non-zero elements
    act_onset = non_zero_indices[0] if non_zero_indices.size > 0 else None
    # time difference btwn activation & stimulus onset portion of activation latency
    stim_act_diff = np.subtract(act_onset - stim_onset)
    # Activation latency: time btwn highest and lowest peak
    peak_time_diff = np.max(evoked_potentials)[0] - np.min(evoked_potentials)[0]

    # Integrated EMG Value
    # TODO: Check that this method makes sense
    rectified_emg = np.abs(evoked_potentials)
    x = np.linspace(0, 24399, 1)
    area = trapezoid(rectified_emg, x)

    # Was there evoked potential?
    potential = np.where(evoked_potentials < threshold, 0, 1)
    
    # Create dataframe to return all data
    df = pd.DataFrame([p2pa, stim_act_diff, peak_time_diff, area, potential], columns=["Peak to Peak Amp, ",
        "Stimulation Time", "Peak Time Difference", "Area Under Peak", "Evoked Potential"])

    return df


In [63]:
data = pd.read_excel('../Data/multichannel_emg_signals.xlsx')
rand = np.random.randint(0, 10, size=24399)
random_nums = pd.DataFrame(rand, columns=['Rand'])
df = pd.concat([data, random_nums], axis=1)
df.head()

,CH1,CH2,CH3,CH4,CH5,Rand
0,-0.000264,-0.000264,-0.000264,-0.000264,-0.000264,2
1,-0.000256,-0.000256,-0.000256,-0.000256,-0.000256,4
2,-0.000254,-0.000254,-0.000254,-0.000254,-0.000254,0
3,-0.000264,-0.000264,-0.000264,-0.000264,-0.000264,2
4,-0.000247,-0.000247,-0.000247,-0.000247,-0.000247,0


In [64]:
p2pa = feature_engineering(df)
print(p2pa)

0        2.000264
1        4.000256
2        0.000254
3        2.000264
4        0.000247
           ...   
24394    3.000193
24395    7.000193
24396    5.000188
24397    9.000177
24398    0.000174
Length: 24399, dtype: float64


### Algorithm 5 - Visualization